# SQL-анализ базы данных сервиса для чтения книг по подписке

**Описание проекта:** Коронавирус застал мир врасплох, изменив привычный порядок вещей. На какое-то время жители городов перестали выходить на улицу, посещать кафе и торговые центры. Зато стало больше времени для книг. Это заметили стартаперы — и бросились создавать приложения для тех, кто любит читать.

**Структура исследования:**
- опишем цели исследования
- исследуем таблицы — выведем первые строки
- сделаем по одному SQL-запросу для решения каждого задания
- выведем результаты каждого запроса в тетрадке
- опишем выводы по каждой из решённых задач

**Описание данных:**

Таблица `books`

Содержит данные о книгах:
- `book_id` — идентификатор книги;
- `author_id` — идентификатор автора;
- `title` — название книги;
- `num_pages` — количество страниц;
- `publication_date` — дата публикации книги;
- `publisher_id` — идентификатор издателя.

Таблица `authors`

Содержит данные об авторах:
- `author_id` — идентификатор автора;
- `author` — имя автора.

Таблица `publishers`

Содержит данные об издательствах:
- `publisher_id` — идентификатор издательства;
- `publisher` — название издательства;

Таблица `ratings`

Содержит данные о пользовательских оценках книг:
- `rating_id` — идентификатор оценки;
- `book_id` — идентификатор книги;
- `username` — имя пользователя, оставившего оценку;
- `rating` — оценка книги.

Таблица reviews

Содержит данные о пользовательских обзорах:
- `review_id` — идентификатор обзора;
- `book_id` — идентификатор книги;
- `username` — имя автора обзора;
- `text` — текст обзора.

## Цель исследования и задачи

**Цель исследования:** Наша компания решила быть на волне и купила крупный сервис для чтения книг по подписке. Наша первая задача как аналитика — проанализировать базу данных. В ней — информация о книгах, издательствах, авторах, а также пользовательские обзоры книг. Эти данные помогут сформулировать ценностное предложение для нового продукта.

**Задачи:**
- посчитать, сколько книг вышло после 1 января 2000 года
- для каждой книги посчитать количество обзоров и среднюю оценку
- определить издательство, которое выпустило наибольшее число книг толще 50 страниц — так мы исключим из анализа брошюры
- определить автора с самой высокой средней оценкой книг — учитываем только книги с 50 и более оценками
- посчитать среднее количество обзоров от пользователей, которые поставили больше 48 оценок

## Подключение к базе данных

In [ ]:
# импортируем библиотеки
import pandas as pd
from sqlalchemy import text, create_engine

In [ ]:
# устанавливаем параметры
db_config = {'user': 'praktikum_student', # имя пользователя
'pwd': 'Sdf4$2;d-d30pp', # пароль
'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
'port': 6432, # порт подключения
'db': 'data-analyst-final-project-db'} # название базы данных
connection_string = 'postgresql://{user}:{pwd}@{host}:{port}/{db}'.format(**db_config)

In [ ]:
# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'})
con = engine.connect()

## Получение общей информации о данных в таблицах

In [ ]:
# Напишем функцию для выполнения запросов
def display(query):
    return pd.io.sql.read_sql(sql=text(query), con=con)

Выведем первые строки таблицы `books`

In [ ]:
books = '''

SELECT *
FROM books
LIMIT 5;

'''

display(books)

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


Выведем первые строки таблицы `authors`

In [ ]:
authors = '''

SELECT *
FROM authors
LIMIT 5;

'''

display(authors)

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


Выведем первые строки таблицы `publishers`

In [ ]:
publishers = '''

SELECT *
FROM publishers
LIMIT 5;

'''

display(publishers)

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


Выведем первые строки таблицы `ratings`

In [ ]:
ratings = '''

SELECT *
FROM ratings
LIMIT 5;

'''

display(ratings)

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


Выведем первые строки таблицы `reviews`

In [ ]:
reviews = '''

SELECT *
FROM reviews
LIMIT 5;

'''

display(reviews)

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


Мы изучили общую информацию о данных, теперь можем приступать к исследовательскому анализу.

## Исследовательский анализ данных

### Посчитаем, сколько книг вышло после 1 января 2000 года

In [ ]:
books_count = '''

SELECT COUNT(*) AS books_count
FROM books
WHERE publication_date > '2000-01-01';

'''

display(books_count)

,books_count
0,819


После 1 января 2000 года вышло **819 книг.**

In [ ]:
all_books = '''

SELECT COUNT(*) AS books_count
FROM books;

'''

display(all_books)

,books_count
0,1000


Всего имеются данные о **1000 книгах.**

### Для каждой книги посчитаем количество обзоров и среднюю оценку

In [ ]:
rating_reviews = '''

SELECT b.title,
       ROUND(AVG(ra.rating), 2) AS avg_rating,
       COUNT(DISTINCT re.review_id) AS total_reviews
FROM books AS b
INNER JOIN ratings AS ra ON ra.book_id = b.book_id
LEFT JOIN reviews AS re ON re.book_id = b.book_id
GROUP BY b.title,
         b.book_id
ORDER BY total_reviews DESC;

'''

display(rating_reviews)

,title,avg_rating,total_reviews
0,Twilight (Twilight #1),3.66,7
1,Water for Elephants,3.98,6
2,The Glass Castle,4.21,6
3,Harry Potter and the Prisoner of Azkaban (Harr...,4.41,6
4,The Curious Incident of the Dog in the Night-Time,4.08,6
...,...,...,...
995,Anne Rice's The Vampire Lestat: A Graphic Novel,3.67,0
996,The Natural Way to Draw,3.00,0
997,The Cat in the Hat and Other Dr. Seuss Favorites,5.00,0
998,Essential Tales and Poems,4.00,0


Для каждой книги было посчитано количество обзоров, и средняя оценка. Топ-5 книг по количеству обзоров:
- *Twilight (Twilight #1)* - **7** обзоров со средней оценкой **3.66**
- *Water for Elephants* - **6** обзоров со средней оценкой **3.98**
- *The Glass Castle* - **6** обзоров со средней оценкой **4.21**
- *Harry Potter and the Prisoner of Azkaban* - **6** обзоров со средней оценкой **4.41**
- *The Curious Incident of the Dog in the Night-Time* - **6** обзоров со средней оценкой **4.08**

### Определим издательство, которое выпустило наибольшее число книг толще 50 страниц

In [ ]:
books_publisher = '''

SELECT p.publisher,
       COUNT(b.book_id) AS books
FROM publishers AS p
JOIN books AS b ON p.publisher_id = b.publisher_id
WHERE b.num_pages > 50
GROUP BY p.publisher_id
ORDER BY books DESC
LIMIT 1;

'''

display(books_publisher)

,publisher,books
0,Penguin Books,42


Издательство, которое выпустило наибольшее число книг толще 50 страниц - *Penguin Books* **(42 книги).**

### Определим автора с самой высокой средней оценкой книг, учитывая только книги с 50 и более оценками

In [ ]:
rating_author = '''

SELECT author,
       AVG(avg_rating) AS average_rating
FROM (SELECT author_id,
             title,
             AVG(rating) AS avg_rating
     FROM books AS b
     LEFT JOIN ratings AS r ON b.book_id = r.book_id
     GROUP BY title, author_id
     HAVING COUNT(rating_id) >= 50) AS query
LEFT JOIN authors AS a ON query.author_id = a.author_id
GROUP BY author
ORDER BY average_rating DESC
LIMIT 1;

'''

display(rating_author)

,author,average_rating
0,J.K. Rowling/Mary GrandPré,4.283844


Автор с самой высокой средней оценкой книг (учитывались только книги с 50 и более оценками) - *J.K. Rowling/Mary GrandPré* **(средняя оценка 4.28).**

### Посчитаем среднее количество обзоров от пользователей, которые поставили больше 48 оценок

In [ ]:
reviews_avg = '''

SELECT ROUND(AVG(count)) AS reviews_avg_count
FROM (SELECT COUNT(text)
     FROM reviews
     FULL OUTER JOIN (SELECT username,
                             COUNT(rating_id) as cnt
                             FROM ratings
                             GROUP BY username) AS cnt_rating
     ON cnt_rating.username = reviews.username
     WHERE cnt > 48
     GROUP BY reviews.username) as count;

'''

display(reviews_avg)

,reviews_avg_count
0,24.0


Среднее количество обзоров от пользователей, которые поставили больше 48 оценок - **24 обзора в среднем.**

## Общие выводы и рекомендации

По результатам проведенного анализа базы данных сервиса для чтения книг по подписке, можно сделать следующие выводы:
1. После 1 января 2000 года вышло **819 книг.** Так как это большинство **(82%),** можно предлагать пользователям эти книги.
2. Для каждой книги было посчитано количество обзоров, и средняя оценка. Топ-5 книг по количеству обзоров:
- *Twilight (Twilight #1)* - **7** обзоров со средней оценкой **3.66**
- *Water for Elephants* - **6** обзоров со средней оценкой **3.98**
- *The Glass Castle* - **6** обзоров со средней оценкой **4.21**
- *Harry Potter and the Prisoner of Azkaban* - **6** обзоров со средней оценкой **4.41**
- *The Curious Incident of the Dog in the Night-Time* - **6** обзоров со средней оценкой **4.08**

Также, наверняка имеются книги с малым количеством обзоров, но с высокой оценкой. Можно рекомендовать их пользователям.

3. Издательство, которое выпустило наибольшее число книг толще 50 страниц - *Penguin Books* **(42 книги).** Можно поощрять такие активные издательства, и рекомендовать их книги.
4. Автор с самой высокой средней оценкой книг (учитывались только книги с 50 и более оценками) - *J.K. Rowling/Mary GrandPré* **(средняя оценка 4.28).** Можно рекомендовать пользователям этого автора, и других популярных авторов.
5. Среднее количество обзоров от пользователей, которые поставили больше 48 оценок - **24 обзора в среднем.** Можно попробовать вовлечь больше пользователей в оценку книг для формирования более точных данных с целью их последующего анализа.